In [ ]:
!conda install pytorch torchvision torchaudio -c pytorch

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/es3773/.conda/envs/hcnn

  added / updated specs:
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.07.19 |       h06a4308_0         131 KB
    cudatoolkit-11.3.1         |       h2bc3f7f_2       815.6 MB
    dataclasses-0.8            |     pyh4f3eec9_6          22 KB
    intel-openmp-2022.0.1      |    h06a4308_3633         8.5 MB
    libuv-1.40.0               |       h7b6447c_0         933 KB
    mkl-2020.2                 |              256       213.9 MB
    mkl-service-2.3.0          |   py36he8ac12f_0          56 KB
    mkl_fft-1.3.0              |   py36h5

In [ ]:

from predify.utils.training import train_pcoders, eval_pcoders

In [2]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset, TrainCleanSoundsDataset, PsychophysicsCleanSoundsDataset

ModuleNotFoundError: No module named 'torch'

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
from pbranchednetwork_conv345 import PBranchedNetwork_Conv345SeparateHP
PNetClass = PBranchedNetwork_Conv345SeparateHP
pnet_name = 'conv345'

# Parameters

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 20
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
train_datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


In [4]:
!nvidia-smi

Thu Jul  7 13:36:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:B2:00.0 Off |                  N/A |
|  0%   28C    P8    19W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.eval()

PBranchedNetwork_Conv345SeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_s

In [8]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up TrainSoundsDataset

In [ ]:
train_dataset = TrainCleanSoundsDataset(train_datafile)

# Set up PsychophysicsSoundsDataset

In [ ]:
f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')

In [ ]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [ ]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [ ]:
def getPsychophysics2017WCleanCochleagrams():
    
    cochleagrams_clean = []
   
    cochleagrams = []
    for batch_ii in range(0,15300,100):
        hdf5_path = '/mnt/smb/locker/issa-locker/users/Erica/cgrams_for_noise_robustness_analysis/PsychophysicsWord2017W_clean/batch_'+str(batch_ii)+'_to_'+str(batch_ii+100)+'.hdf5'
        with h5py.File(hdf5_path, 'r') as f_in:
            cochleagrams += list(f_in['data'])

    return cochleagrams
clean_in = getPsychophysics2017WCleanCochleagrams()
clean_in = np.array(clean_in)

In [ ]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

In [ ]:
orig_dset = []
for _orig_dset in f_metadata['orig_dset']:
    _orig_dset = str(_orig_dset, 'utf-8')
    _orig_dset = 'WSJ' if 'WSJ' in _orig_dset else 'Timit'
    orig_dset.append(_orig_dset)
orig_dset = np.array(orig_dset)

In [ ]:
psycho_dataset = PsychophysicsCleanSoundsDataset(
    clean_in, labels, orig_dset, exclude_timit=True
    )

In [ ]:
del orig_dset
del clean_in
del labels

# Merge Datasets

In [ ]:
full_dataset = CleanSoundsDataset(train_dataset, psycho_dataset)
del train_dataset
del psycho_dataset
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

torch.Size([55188, 1, 164, 400])


# Set up checkpoints and tensorboards

In [ ]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [20/49669]	Loss: 11970782.0000
Training Epoch: 1 [40/49669]	Loss: 7230903.0000
Training Epoch: 1 [60/49669]	Loss: 4840701.5000
Training Epoch: 1 [80/49669]	Loss: 2625204.5000
Training Epoch: 1 [100/49669]	Loss: 2356654.5000
Training Epoch: 1 [120/49669]	Loss: 2885562.7500
Training Epoch: 1 [140/49669]	Loss: 2969443.2500
Training Epoch: 1 [160/49669]	Loss: 3177833.2500
Training Epoch: 1 [180/49669]	Loss: 3565537.2500
Training Epoch: 1 [200/49669]	Loss: 3896911.5000
Training Epoch: 1 [220/49669]	Loss: 4000930.0000
Training Epoch: 1 [240/49669]	Loss: 2949415.7500
Training Epoch: 1 [260/49669]	Loss: 3125396.7500
Training Epoch: 1 [280/49669]	Loss: 2520996.7500
Training Epoch: 1 [300/49669]	Loss: 2221031.2500
Training Epoch: 1 [320/49669]	Loss: 1895894.5000
Training Epoch: 1 [340/49669]	Loss: 1716454.0000
Training Epoch: 1 [360/49669]	Loss: 1640568.8750
Training Epoch: 1 [380/49669]	Loss: 2086494.2500
Training Epoch: 1 [400/49669]	Loss: 1835595.6250
Training Epoch: 1 [420/

Training Epoch: 1 [3380/49669]	Loss: 312975.1250
Training Epoch: 1 [3400/49669]	Loss: 258586.9531
Training Epoch: 1 [3420/49669]	Loss: 286245.7188
Training Epoch: 1 [3440/49669]	Loss: 238968.1719
Training Epoch: 1 [3460/49669]	Loss: 343129.0000
Training Epoch: 1 [3480/49669]	Loss: 304746.6875
Training Epoch: 1 [3500/49669]	Loss: 254322.7031
Training Epoch: 1 [3520/49669]	Loss: 261568.3125
Training Epoch: 1 [3540/49669]	Loss: 316277.0312
Training Epoch: 1 [3560/49669]	Loss: 236765.6875
Training Epoch: 1 [3580/49669]	Loss: 230135.2188
Training Epoch: 1 [3600/49669]	Loss: 327814.6875
Training Epoch: 1 [3620/49669]	Loss: 294868.1875
Training Epoch: 1 [3640/49669]	Loss: 277500.7812
Training Epoch: 1 [3660/49669]	Loss: 258375.6875
Training Epoch: 1 [3680/49669]	Loss: 301420.0938
Training Epoch: 1 [3700/49669]	Loss: 299485.3125
Training Epoch: 1 [3720/49669]	Loss: 314791.6562
Training Epoch: 1 [3740/49669]	Loss: 294830.8125
Training Epoch: 1 [3760/49669]	Loss: 310814.1250
Training Epoch: 1 [3

Training Epoch: 1 [6740/49669]	Loss: 167596.2188
Training Epoch: 1 [6760/49669]	Loss: 245383.0000
Training Epoch: 1 [6780/49669]	Loss: 150397.2812
Training Epoch: 1 [6800/49669]	Loss: 208541.5469
Training Epoch: 1 [6820/49669]	Loss: 172541.8438
Training Epoch: 1 [6840/49669]	Loss: 189706.3438
Training Epoch: 1 [6860/49669]	Loss: 200592.7344
Training Epoch: 1 [6880/49669]	Loss: 217584.1719
Training Epoch: 1 [6900/49669]	Loss: 186651.1250
Training Epoch: 1 [6920/49669]	Loss: 223817.9062
Training Epoch: 1 [6940/49669]	Loss: 229878.6875
Training Epoch: 1 [6960/49669]	Loss: 224099.9688
Training Epoch: 1 [6980/49669]	Loss: 177144.4531
Training Epoch: 1 [7000/49669]	Loss: 212799.0625
Training Epoch: 1 [7020/49669]	Loss: 150062.4844
Training Epoch: 1 [7040/49669]	Loss: 182453.2969
Training Epoch: 1 [7060/49669]	Loss: 194234.5938
Training Epoch: 1 [7080/49669]	Loss: 198950.7656
Training Epoch: 1 [7100/49669]	Loss: 162530.5625
Training Epoch: 1 [7120/49669]	Loss: 154839.9219
Training Epoch: 1 [7

Training Epoch: 1 [10100/49669]	Loss: 175074.0781
Training Epoch: 1 [10120/49669]	Loss: 156310.4844
Training Epoch: 1 [10140/49669]	Loss: 182808.0000
Training Epoch: 1 [10160/49669]	Loss: 134538.4688
Training Epoch: 1 [10180/49669]	Loss: 149353.0781
Training Epoch: 1 [10200/49669]	Loss: 152016.5781
Training Epoch: 1 [10220/49669]	Loss: 181797.4844
Training Epoch: 1 [10240/49669]	Loss: 147133.5625
Training Epoch: 1 [10260/49669]	Loss: 168790.7031
Training Epoch: 1 [10280/49669]	Loss: 160441.6250
Training Epoch: 1 [10300/49669]	Loss: 141556.6875
Training Epoch: 1 [10320/49669]	Loss: 140596.4375
Training Epoch: 1 [10340/49669]	Loss: 163964.7812
Training Epoch: 1 [10360/49669]	Loss: 158340.6094
Training Epoch: 1 [10380/49669]	Loss: 147280.7500
Training Epoch: 1 [10400/49669]	Loss: 122706.9609
Training Epoch: 1 [10420/49669]	Loss: 155335.7969
Training Epoch: 1 [10440/49669]	Loss: 154708.5625
Training Epoch: 1 [10460/49669]	Loss: 125826.9531
Training Epoch: 1 [10480/49669]	Loss: 175668.8125


...

Training Epoch: 4 [9020/49669]	Loss: 2426.9719
Training Epoch: 4 [9040/49669]	Loss: 2409.6592
Training Epoch: 4 [9060/49669]	Loss: 2794.5039
Training Epoch: 4 [9080/49669]	Loss: 2458.7739
Training Epoch: 4 [9100/49669]	Loss: 2565.4231
Training Epoch: 4 [9120/49669]	Loss: 2570.5325
Training Epoch: 4 [9140/49669]	Loss: 2687.2554
Training Epoch: 4 [9160/49669]	Loss: 2689.8330
Training Epoch: 4 [9180/49669]	Loss: 2520.8613
Training Epoch: 4 [9200/49669]	Loss: 2465.6174
Training Epoch: 4 [9220/49669]	Loss: 2229.4631
Training Epoch: 4 [9240/49669]	Loss: 2815.0632
Training Epoch: 4 [9260/49669]	Loss: 2366.1411
Training Epoch: 4 [9280/49669]	Loss: 2579.9746
Training Epoch: 4 [9300/49669]	Loss: 2540.1943
Training Epoch: 4 [9320/49669]	Loss: 2526.4028
Training Epoch: 4 [9340/49669]	Loss: 2703.2065
Training Epoch: 4 [9360/49669]	Loss: 2069.0654
Training Epoch: 4 [9380/49669]	Loss: 2524.0969
Training Epoch: 4 [9400/49669]	Loss: 2314.1431
Training Epoch: 4 [9420/49669]	Loss: 2634.6892
Training Epoc

Training Epoch: 4 [12460/49669]	Loss: 2879.8826
Training Epoch: 4 [12480/49669]	Loss: 2499.7915
Training Epoch: 4 [12500/49669]	Loss: 2393.2312
Training Epoch: 4 [12520/49669]	Loss: 2441.0781
Training Epoch: 4 [12540/49669]	Loss: 2621.0706
Training Epoch: 4 [12560/49669]	Loss: 2451.7986
Training Epoch: 4 [12580/49669]	Loss: 2564.4316
Training Epoch: 4 [12600/49669]	Loss: 2695.3530
Training Epoch: 4 [12620/49669]	Loss: 2702.3518
Training Epoch: 4 [12640/49669]	Loss: 2760.1045
Training Epoch: 4 [12660/49669]	Loss: 2304.5369
Training Epoch: 4 [12680/49669]	Loss: 2493.5432
Training Epoch: 4 [12700/49669]	Loss: 2641.8398
Training Epoch: 4 [12720/49669]	Loss: 2296.9170
Training Epoch: 4 [12740/49669]	Loss: 2577.6697
Training Epoch: 4 [12760/49669]	Loss: 2597.1707
Training Epoch: 4 [12780/49669]	Loss: 2569.5972
Training Epoch: 4 [12800/49669]	Loss: 2463.9219
Training Epoch: 4 [12820/49669]	Loss: 2744.6292
Training Epoch: 4 [12840/49669]	Loss: 2680.9421
Training Epoch: 4 [12860/49669]	Loss: 20

Training Epoch: 4 [15880/49669]	Loss: 2399.4045
Training Epoch: 4 [15900/49669]	Loss: 2814.6943
Training Epoch: 4 [15920/49669]	Loss: 2508.5586
Training Epoch: 4 [15940/49669]	Loss: 2627.3440
Training Epoch: 4 [15960/49669]	Loss: 2202.7903
Training Epoch: 4 [15980/49669]	Loss: 2701.8342
Training Epoch: 4 [16000/49669]	Loss: 2620.4617
Training Epoch: 4 [16020/49669]	Loss: 2263.4326
Training Epoch: 4 [16040/49669]	Loss: 2277.8313
Training Epoch: 4 [16060/49669]	Loss: 2742.1226


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Training Epoch: 5 [39880/49669]	Loss: 2258.8884
Training Epoch: 5 [39900/49669]	Loss: 2052.2622
Training Epoch: 5 [39920/49669]	Loss: 2395.2161
Training Epoch: 5 [39940/49669]	Loss: 2269.2053
